In [1]:
from sentence_transformers import SentenceTransformer
import pandas as pd
import numpy as np
import pickle
import torch
import sys
import os

parent_dir = os.path.abspath(os.path.join(os.getcwd(), '..'))
src_path = os.path.join(parent_dir, 'src')
if src_path not in sys.path:
    sys.path.append(src_path)
    
from loss import OrdinalLoss
from utils import compute_graded_metrics
from transformers import AutoTokenizer, AutoModel
from torch.utils.data import Dataset, DataLoader
import torch.nn as nn
from torch.optim import AdamW
from tqdm.auto import tqdm

/opt/homebrew/Caskroom/miniconda/base/envs/py311/lib/python3.11/site-packages/transformers/utils/generic.py:441: FutureWarning: `torch.utils._pytree._register_pytree_node` is deprecated. Please use `torch.utils._pytree.register_pytree_node` instead.
  _torch_pytree._register_pytree_node(
/opt/homebrew/Caskroom/miniconda/base/envs/py311/lib/python3.11/site-packages/transformers/utils/generic.py:309: FutureWarning: `torch.utils._pytree._register_pytree_node` is deprecated. Please use `torch.utils._pytree.register_pytree_node` instead.
  _torch_pytree._register_pytree_node(


In [20]:
MODEL_NAME = "microsoft/deberta-v3-base"
MAX_LEN = 512   # tokens
BATCH_SIZE = 8
EPOCHS = 4
LEARNING_RATE = 2e-5
NUM_CLASSES = 4
ALPHA_ORDINAL = 1.5

DEVICE = "cuda" if torch.cuda.is_available() else "cpu"
if torch.backends.mps.is_available():
    DEVICE = torch.device("mps")
    print("Using MPS (Apple Silicon GPU)")

print("Device:", DEVICE)

Using MPS (Apple Silicon GPU)
Device: mps


In [3]:
!pip install -r ../requirements.txt

In [4]:
!pip install sentencepiece

In [21]:
# Load PKL datasets

train_df = pickle.load(open("../data/processed/train.pkl", "rb"))
val_df   = pickle.load(open("../data/processed/val.pkl", "rb"))
test_df  = pickle.load(open("../data/processed/test.pkl", "rb"))

print("Train size:", len(train_df))
print("Val size:", len(val_df))
print("Test size:", len(test_df))

print(test_df)

Train size: 11972
Val size: 1605
Test size: 1036
      users                                               text  sentiment  \
0        24  If there is a god it’s not the one that you th...  Indicator   
1        24  I just wish it was ok to end our lives if that...   Ideation   
2        24  Sometimes suicide is a very appropriate reacti...   Ideation   
3        24  I’m exhausted but I can’t fall asleep the ment...   Ideation   
4        24  The pain just gets worse. How does it keep get...   Ideation   
...     ...                                                ...        ...   
1031   1235  I want to kill myself but I'm scared. And I do...   Ideation   
1032   1253  Yeah, that is how it is sometimes. I'll be gon...   Behavior   
1033   1253  haha, noone's going to see this but here goes ...   Behavior   
1034   1259  Just venting I tried to kill myself about two ...    Attempt   
1035   1261  I did it.. I made a reddit account. Purely to ...  Indicator   

            time        ti

/var/folders/qp/cfpchbnx73v1pnmhgwclp3m40000gn/T/ipykernel_69520/4138350335.py:3: DeprecationWarning: numpy.core.numeric is deprecated and has been renamed to numpy._core.numeric. The numpy._core namespace contains private NumPy internals and its use is discouraged, as NumPy internals can change without warning in any release. In practice, most real-world usage of numpy.core is to access functionality in the public NumPy API. If that is the case, use the public NumPy API. If not, you are using NumPy internals. If you would still like to access an internal attribute, use numpy._core.numeric._frombuffer.
  train_df = pickle.load(open("../data/processed/train.pkl", "rb"))
/var/folders/qp/cfpchbnx73v1pnmhgwclp3m40000gn/T/ipykernel_69520/4138350335.py:4: DeprecationWarning: numpy.core.numeric is deprecated and has been renamed to numpy._core.numeric. The numpy._core namespace contains private NumPy internals and its use is discouraged, as NumPy internals can change without warning in any re

In [22]:
def add_timestamp_features(df):

    df["timestamp_dt"] = pd.to_datetime(df["timestamp_dt"])
    df["hour"] = df["timestamp_dt"].dt.hour
    df["day_of_week"] = df["timestamp_dt"].dt.dayofweek

    # Sort by user & time
    df = df.sort_values(["users", "timestamp_dt"])

    # === Generated by Gemini ===
    # Time difference to previous post
    df["time_gap"] = df.groupby("users")["timestamp_dt"].diff().dt.total_seconds()
    df["time_gap"] = df["time_gap"].fillna(0)

    # Time since user's first post
    df["time_since_start"] = (
        df["timestamp_dt"] - df.groupby("users")["timestamp_dt"].transform("min")
    ).dt.total_seconds()
    # === End of Gemini-generated block ===
    
    df["normalized_gap"] = np.log1p(df["time_gap"])  # log(1 + seconds)

    return df

train_df = add_timestamp_features(train_df)
val_df   = add_timestamp_features(val_df)
test_df  = add_timestamp_features(test_df)

print(test_df)

      users                                               text  sentiment  \
0        24  If there is a god it’s not the one that you th...  Indicator   
1        24  I just wish it was ok to end our lives if that...   Ideation   
2        24  Sometimes suicide is a very appropriate reacti...   Ideation   
3        24  I’m exhausted but I can’t fall asleep the ment...   Ideation   
4        24  The pain just gets worse. How does it keep get...   Ideation   
...     ...                                                ...        ...   
1031   1235  I want to kill myself but I'm scared. And I do...   Ideation   
1032   1253  Yeah, that is how it is sometimes. I'll be gon...   Behavior   
1033   1253  haha, noone's going to see this but here goes ...   Behavior   
1034   1259  Just venting I tried to kill myself about two ...    Attempt   
1035   1261  I did it.. I made a reddit account. Purely to ...  Indicator   

            time        timestamp_dt  label_ordinal  hour  day_of_week  \
0

In [23]:
# DeBERTa Tokenization

tokenizer = AutoTokenizer.from_pretrained(MODEL_NAME, use_fast=False)

train_encodings = tokenizer(train_df["text"].tolist(), truncation=True, padding="max_length", max_length=MAX_LEN, return_tensors="pt")
val_encodings   = tokenizer(val_df["text"].tolist(), truncation=True, padding="max_length", max_length=MAX_LEN, return_tensors="pt")
test_encodings  = tokenizer(test_df["text"].tolist(), truncation=True, padding="max_length", max_length=MAX_LEN, return_tensors="pt")

print(test_df)

/opt/homebrew/Caskroom/miniconda/base/envs/py311/lib/python3.11/site-packages/huggingface_hub/file_download.py:942: FutureWarning: `resume_download` is deprecated and will be removed in version 1.0.0. Downloads always resume when possible. If you want to force a new download, use `force_download=True`.
  warnings.warn(


      users                                               text  sentiment  \
0        24  If there is a god it’s not the one that you th...  Indicator   
1        24  I just wish it was ok to end our lives if that...   Ideation   
2        24  Sometimes suicide is a very appropriate reacti...   Ideation   
3        24  I’m exhausted but I can’t fall asleep the ment...   Ideation   
4        24  The pain just gets worse. How does it keep get...   Ideation   
...     ...                                                ...        ...   
1031   1235  I want to kill myself but I'm scared. And I do...   Ideation   
1032   1253  Yeah, that is how it is sometimes. I'll be gon...   Behavior   
1033   1253  haha, noone's going to see this but here goes ...   Behavior   
1034   1259  Just venting I tried to kill myself about two ...    Attempt   
1035   1261  I did it.. I made a reddit account. Purely to ...  Indicator   

            time        timestamp_dt  label_ordinal  hour  day_of_week  \
0

In [24]:
class TimeAwareDataset(Dataset):
    def __init__(self, encodings, df):
        self.input_ids = encodings['input_ids']
        self.attention_mask = encodings['attention_mask']
        self.targets = torch.tensor(df["label_ordinal"].values, dtype=torch.long)
        
        # Extract and prepare time features: hour, day of week, time gap, normalized gap
        # Ensure the order is consistent with the model's expectations
        time_feats = df[["hour", "day_of_week", "time_gap", "normalized_gap"]].values
        self.time_feats = torch.tensor(time_feats, dtype=torch.float32)

    # Will be used when calling DataLoader   
    def __len__(self): 
        return len(self.targets)

    # Will be used when calling DataLoader
    def __getitem__(self, idx):
        return {
            'input_ids': self.input_ids[idx],
            'attention_mask': self.attention_mask[idx],
            'time_feats': self.time_feats[idx],
            'targets': self.targets[idx]
        }

# Create Datasets & DataLoaders
train_dataset = TimeAwareDataset(train_encodings, train_df)
val_dataset = TimeAwareDataset(val_encodings, val_df)
test_dataset = TimeAwareDataset(test_encodings, test_df)

train_loader = DataLoader(train_dataset, batch_size=BATCH_SIZE, shuffle=True)
val_loader = DataLoader(val_dataset, batch_size=BATCH_SIZE, shuffle=False)
test_loader = DataLoader(test_dataset, batch_size=BATCH_SIZE, shuffle=False)

print("DataLoaders created with BATCH_SIZE=", BATCH_SIZE)

DataLoaders created with BATCH_SIZE= 8


In [25]:
class TimeAwareOrdinalModel(nn.Module):
    def __init__(self, model_name, time_feat_dim, num_classes, lstm_hidden_size=256, dropout_rate=0.3):
        super(TimeAwareOrdinalModel, self).__init__()
        
        # 1. DeBERTa Pre-trained model
        self.text_model = AutoModel.from_pretrained(model_name)
        embed_dim = self.text_model.config.hidden_size # DeBERTa Base 默認是 768
        
        # 2. BiLSTM layer: Used to process DeBERTa sequence output
        # Input size is 768 (DeBERTa hidden layer size)
        self.bilstm = nn.LSTM(
            input_size=embed_dim,
            hidden_size=lstm_hidden_size, # 256
            num_layers=1,
            bidirectional=True, # bi-direction
            batch_first=True
        )
        
        # BiLSTM output dimension: 2 * lstm_hidden_size (because it is bidirectional)
        bilstm_output_dim = lstm_hidden_size * 2
        
        # 3. Total input dimension of merged features
        # Total input dimension = BiLSTM output + temporal features (4)
        total_input_dim = bilstm_output_dim + time_feat_dim
        
        # 4. Classifier header
        self.fc1 = nn.Linear(total_input_dim, lstm_hidden_size)
        self.relu = nn.ReLU()
        self.dropout = nn.Dropout(dropout_rate)
        self.output_layer = nn.Linear(lstm_hidden_size, num_classes)
        
    def forward(self, input_ids, attention_mask, time_feats):
        
        # 1. Get DeBERTa sequence output
        # text_output.last_hidden_state shape: (B, MAX_LEN, embed_dim=768)
        text_output = self.text_model(input_ids=input_ids, attention_mask=attention_mask)
        sequence_output = text_output.last_hidden_state
        
        # 2. Passing to BiLSTM
        # Inputting the sequence into BiLSTM
        lstm_output, _ = self.bilstm(sequence_output)
        
        # 3. Extracting the output of a BiLSTM: Typically, we take the output of the last time step of the sequence (or the sequence after max pooling/average pooling).
        # Here we take the output of the last time step (the last token) of the BiLSTM.
        # lstm_output shape: (B, MAX_LEN, 2 * lstm_hidden_size)
        lstm_pooled = lstm_output[:, 0, :] # Extract the BiLSTM output of the first token (similar to [CLS]) from the sequence.
        
        # 4. Combined features: [BiLSTM output, temporal features] 
        combined_features = torch.cat((lstm_pooled, time_feats), dim=1)
        
        # 5. Passed to classification layer
        out = self.fc1(combined_features)
        out = self.relu(out)
        out = self.dropout(out)
        logits = self.output_layer(out)
        
        return logits

# Instantiation of model and loss function
TIME_FEAT_DIM = 4 # hour, day_of_week, time_gap, normalized_gap
model = TimeAwareOrdinalModel(MODEL_NAME, TIME_FEAT_DIM, NUM_CLASSES).to(DEVICE)
loss_fn = OrdinalLoss(alpha=ALPHA_ORDINAL, num_classes=NUM_CLASSES, device=DEVICE)
optimizer = AdamW(model.parameters(), lr=LEARNING_RATE)

print("Model, Loss, and Optimizer initialized.")

/opt/homebrew/Caskroom/miniconda/base/envs/py311/lib/python3.11/site-packages/huggingface_hub/file_download.py:942: FutureWarning: `resume_download` is deprecated and will be removed in version 1.0.0. Downloads always resume when possible. If you want to force a new download, use `force_download=True`.
  warnings.warn(


Using MPS (Apple Silicon GPU)
Model, Loss, and Optimizer initialized.


In [32]:
def train_epoch(model, dataloader, loss_fn, optimizer, device):
    model.train()
    total_loss = 0
    
    for batch in tqdm(dataloader, desc="Training"):
        input_ids = batch['input_ids'].to(device)
        attention_mask = batch['attention_mask'].to(device)
        time_feats = batch['time_feats'].to(device)
        targets = batch['targets'].to(device)
        
        optimizer.zero_grad()
        logits = model(input_ids, attention_mask, time_feats)
        
        # OrdinalLoss returns a shape of (B,) which needs to be calculated using mean().
        loss = loss_fn(logits, targets)
        loss.mean().backward()
        optimizer.step()
        
        total_loss += loss.mean().item()
        
    return total_loss / len(dataloader)

In [33]:
def evaluate(model, dataloader, device):
    model.eval()
    all_preds = []
    all_targets = []
    
    with torch.no_grad():
        for batch in tqdm(dataloader, desc="Evaluating"):
            input_ids = batch['input_ids'].to(device)
            attention_mask = batch['attention_mask'].to(device)
            time_feats = batch['time_feats'].to(device)
            targets = batch['targets'].to(device)
            
            logits = model(input_ids, attention_mask, time_feats)
            
            # Predicted category (index with the highest logit count)
            preds = torch.argmax(logits, dim=1)
            
            all_preds.extend(preds.cpu().tolist())
            all_targets.extend(targets.cpu().tolist())

            metrics = compute_graded_metrics(all_targets, all_preds)
    return metrics

print("Training and evaluation functions defined.")

Training and evaluation functions defined.


In [34]:
best_f1 = 0
MODEL_SAVE_PATH = "time_aware_deberta_ordinal_best.pt"

for epoch in range(EPOCHS):
    print(f"\n--- Epoch {epoch+1}/{EPOCHS} ---")
    
    # Train
    avg_train_loss = train_epoch(model, train_loader, loss_fn, optimizer, DEVICE)
    
    # Evaluate
    val_metrics = evaluate(model, val_loader, DEVICE)
    
    print(f"Train Loss: {avg_train_loss:.4f}")
    print(f"Validation Metrics: Graded Precision={val_metrics['graded_precision']}, Graded Recall={val_metrics['graded_recall']}, Graded F1={val_metrics['graded_f1']}")
    
    # Save the best model
    if val_metrics['graded_f1'] > best_f1:
        best_f1 = val_metrics['graded_f1']
        torch.save(model.state_dict(), MODEL_SAVE_PATH)
        print("Model saved! New best F1.")

print("Training complete.")


--- Epoch 1/4 ---


Training:   0%|          | 0/1497 [00:00<?, ?it/s]

RuntimeError: MPS backend out of memory (MPS allocated: 17.68 GiB, other allocations: 393.25 MiB, max allowed: 18.13 GiB). Tried to allocate 96.00 MiB on private pool. Use PYTORCH_MPS_HIGH_WATERMARK_RATIO=0.0 to disable upper limit for memory allocations (may cause system failure).

In [ ]:
print("\n--- Final Test Set Evaluation ---")

# Load the best model
try:
    model.load_state_dict(torch.load(MODEL_SAVE_PATH, map_location=DEVICE))
    print(f"Loaded best model from {MODEL_SAVE_PATH}")
except FileNotFoundError:
    print(f"Warning: Best model file {MODEL_SAVE_PATH} not found. Using final epoch model.")

# Evaluate test set
test_metrics = evaluate(model, test_loader, DEVICE)

print("\nTest Set Results:")
print(f"Graded Precision: {test_metrics['graded_precision']}")
print(f"Graded Recall: {test_metrics['graded_recall']}")
print(f"Graded F1 Score: {test_metrics['graded_f1']}")